# Generate push4/push8 videos

This notebook contains the code needed to reproduce the push4 and push8 videos created for the supplementary material.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# add the egreedy module to the path (one directory up from this)
import sys, os
sys.path.append(os.path.realpath(os.path.pardir))

In [2]:
problem_names = ['push4', 'push8']

method_names = ['LHS', 'Explore', 'EI', 'PI', 'UCB', 'PFRandom',
                'eRandom_eps0.1', 'eFront_eps0.1', 'Exploit']

# Generate individual videos showing the quality of random samples

In [3]:
from egreedy import test_problems

save_dir = r'C:\temp\push_images'

In [ ]:
for problem_name in problem_names:

    for run_no in [1, 2, 3]:
        # load training points and problem instance info
        training_data = f'../training_data/{problem_name:}_{run_no:}.npz'
        with np.load(training_data, allow_pickle=True) as data:
            Xtr = data['arr_0']
            Ytr = data['arr_1']
            f_optional_arguments = data['arr_2'].item()

        # instantiate function
        f_class = getattr(test_problems, problem_name)
        f = f_class(**f_optional_arguments)

        x = Xtr[0]

        save_dir_path = os.path.join(save_dir, f'{problem_name}_{run_no}_LHS')
        if not os.path.exists(save_dir_path):
            os.makedirs(save_dir_path)

        f(x, {'show': True,
              'save': True,
              'save_every': 1,
              'save_dir': save_dir_path,
              'save_prefix': f'{problem_name}_{run_no}'
             })

# Repeat for the best solutions to a number of problem instances

In [ ]:
for problem_name in ['push4', 'push8']:
    for run_no in [4, 18, 23, 24, 25, 43, 37, 38]:
        # load the problem instance params
        training_data = f'../training_data/{problem_name:}_{run_no:}.npz'
        with np.load(training_data, allow_pickle=True) as data:
            f_optional_arguments = data['arr_2'].item()

        # instantiate function
        f_class = getattr(test_problems, problem_name)
        f = f_class(**f_optional_arguments)
        
        # find the best run
        best_y = np.inf
        best_x = None

        for method_name in method_names:
            save_file = f'../results_paper/{problem_name:}_{run_no:}_250_{method_name:}.npz'
            with np.load(save_file, allow_pickle=True) as data:
                Xtr = data['Xtr']
                Ytr = data['Ytr'].flat
                
            argmin = np.argmin(Ytr)
            if Ytr[argmin] < best_y:
                best_y = Ytr[argmin]
                best_x = Xtr[argmin, :]
                
        # write the images
        save_dir_path = os.path.join(save_dir, f'{problem_name}_{run_no}_best')
        if not os.path.exists(save_dir_path):
            os.makedirs(save_dir_path)

        ret = f(best_x, {'show': True,
                         'save': True,
                         'save_every': 1,
                         'save_dir': save_dir_path,
                         'save_prefix': f'{problem_name}_{run_no}'
                         })

# write all the images to video files using imageio

In [ ]:
import glob
from imageio import get_writer, imread

folders = [path for path in glob.glob(os.path.join(save_dir, '*')) 
           if os.path.isdir(path)]

folders = sorted(folders)

for problem_folder in folders:

    problem_name = os.path.basename(problem_folder)
    print(problem_folder, problem_name)
    
    # read in video
    files = glob.glob(os.path.join(problem_folder, '*.png'))
    files = sorted(files, key=lambda x: int(x.split('_')[-2]))

    save_fname = os.path.join(save_dir, f'{problem_name}.mp4')
    
    with get_writer(save_fname, fps=30) as w:
        for file in files:
            I = imread(file)
            w.append_data(imread(file))

# Create two videos showing the optimisation improvement over a run

In [33]:
import cv2
import glob
from imageio import get_writer, imread

iterations = [0, 49, 99, 199, 249]

for problem_name, run_no in [['push4', 15], 
                             ['push8', 18]]:
    # load the problem instance params
    training_data = f'../training_data/{problem_name:}_{run_no:}.npz'
    with np.load(training_data, allow_pickle=True) as data:
        f_optional_arguments = data['arr_2'].item()

    # instantiate function
    f_class = getattr(test_problems, problem_name)
    f = f_class(**f_optional_arguments)

    # find the best run
    best_Ytr = np.inf
    best_Xtr = None

    for method_name in method_names:
        save_file = f'../results_paper/{problem_name:}_{run_no:}_250_{method_name:}.npz'
        with np.load(save_file, allow_pickle=True) as data:
            Xtr = data['Xtr']
            Ytr = data['Ytr'].ravel()

        argmin = np.argmin(Ytr)
        if Ytr[argmin] < np.min(best_Ytr):
            best_Ytr = Ytr
            best_Xtr = Xtr
            
    save_fname = os.path.join(save_dir, f'{problem_name}_{run_no}_combined.mp4')
    
    with get_writer(save_fname, fps=100) as w:
        sorted_inds = np.argsort(best_Ytr)[::-1]
        for i in iterations:
            # write the images
            save_dir_path = os.path.join(save_dir, f'{problem_name}_{run_no}_{i}')
            if not os.path.exists(save_dir_path):
                os.makedirs(save_dir_path)
            ret = f(best_Xtr[sorted_inds[i], :], {'show': True,
                                    'save': True,
                                    'save_every': 1,
                                    'save_dir': save_dir_path,
                                    'save_prefix': f'{problem_name}_{run_no}'
                                    })
            print(i, ret, best_Ytr[sorted_inds[i]])

            # read in video
            files = glob.glob(os.path.join(save_dir_path, '*.png'))
            files = sorted(files, key=lambda x: int(x.split('_')[-2]))
        
            for file in files:
                I = imread(file)
                cv2.putText(I, f'Iteration {i+1:d}', (10, 740), 
                            cv2.FONT_HERSHEY_COMPLEX,
                            1, (0, 0, 0), 1, cv2.LINE_AA)
                w.append_data(I)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (750, 750) to (752, 752) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


0 [26.26065088] 26.260650881577234
49 [4.64730593] 4.6473059286451885
99 [1.74190446] 1.74190446051895
199 [0.84050055] 0.8405005481257952
249 [0.01313448] 0.01313448110158422


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (750, 750) to (752, 752) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


0 [35.76675201] 35.766752006637745
49 [9.80116746] 9.80116745917844
99 [6.21338043] 6.213380428986013
199 [1.42174986] 1.4217498551129917
249 [0.12470443] 0.12470442504557139


In [19]:
# read in video
files = glob.glob(os.path.join(save_dir_path, '*.png'))
files = sorted(files, key=lambda x: int(x.split('_')[-2]))
with get_writer(save_fname, fps=30) as w:
    for file in files:
        I = imread(file)
        cv2.putText(I, f'Iteration {i+1:d}', (10, 740), 
                    cv2.FONT_HERSHEY_COMPLEX,
                    1, (0, 0, 0), 1, cv2.LINE_AA)

        w.append_data(I)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (750, 750) to (752, 752) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
